# Objective:

### Use MNIST dataset and apply PCA to find out the impact on the model training time and also model performance
### The work is taken from https://github.com/mGalarnyk/Python_Tutorials/blob/master/Sklearn/PCA/PCA_to_Speed-up_Machine_Learning_Algorithms.ipynb

In [14]:
# Setup
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml


#Download and Load the Data


In [15]:
from sklearn.datasets import fetch_openml
#get MNIST data
mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings

X, y = mnist["data"], mnist["target"]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


# Split data into train/test

In [16]:
# Write a code to split your dataset into 80/20 dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# View Data Dimension

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((56000, 784), (14000, 784), (56000,), (14000,))

#Standardizing the Data¶

Since PCA yields a feature subspace that maximizes the variance along the axes, it makes sense to standardize the data, especially, if it was measured on different scales.

Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual feature do not more or less look like standard normally distributed data

Notebook going over the importance of feature Scaling: http://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py


In [18]:
from pandas.core.tools.datetimes import Scalar
from sklearn.preprocessing import StandardScaler
#define scaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train);

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test =  scaler.transform(X_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((56000, 784), (14000, 784), (56000,), (14000,))

In [19]:
# In case you want to see how the scaled number would look like, you can uncomment below lines
from scipy.stats import describe
describe(X_train)[1]

(array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.00457315, -0.00595681, -0.00422581,
        -0.00422581,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.00422581, -0.00644617, -0.00859209,
        -0.01091681, -0.01355956, -0.01941039, -0.02448325, -0.02895702,
        -0.0301599 , -0.03233274, -0.0332658 , -0.02905717, -0.02870756,
        -0.02701646, -0.02273783, -0.01762933, -0.01644529, -0.01059506,
        -0.00764366, -0.00422581,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.00422581,
        -0.00809245, -0.01071301, -0.01764173, -0.02844364, -0.03769862,
        -0.0536141 , -0.0693414 , -0.0867893 , -0.1

# Build PCA

In [50]:
from sklearn.decomposition import PCA

#define PCA with 0.9 variance to capture
pca = PCA(0.95)

#use it to fit X_train
pca.fit(X_train)

InvalidParameterError: ignored

# Look at components

In [47]:
pca.n_components_


1

#Apply the mapping (transform) to both the training set and the test set.



In [48]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

#Build a linear model and measure model fitting period.

In [49]:
# Build a linear model using below parameters
from sklearn.linear_model import LogisticRegression

#solver = 'lbfgs'
# multi_class ='auto'

#define the LogisticRegression model
logisticRegr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

import datetime
start= datetime.datetime.now()

# Fit the model
logisticRegr.fit(X_train_pca, y_train)

end= datetime.datetime.now()

print(end-start)


0:00:01.402026


In [44]:

logisticRegr.predict(X_train_pca[0].reshape(1,-1))


array([5], dtype=int8)

#Measuring Model Performance

In [45]:
score = logisticRegr.score(X_test_pca, y_test)
print(score)

0.9216428571428571


In [52]:
#Replace numbers with yours
pd.DataFrame(data = [
                     [.95, 330, 118, .9216],
                     [.90, 235, 85, .9225],
                     [.85, 184, 70, .9223]],
             columns = ['Variance Retained',
                      'Number of Components',
                      'Time (seconds)',
                      'Accuracy'])

,Variance Retained,Number of Components,Time (seconds),Accuracy
0,0.95,330,118,0.9216
1,0.90,235,85,0.9225
2,0.85,184,70,0.9223
